## Kings County Linear Regretion Model

## Overview

This notebook is an implimentation of a liner regretion model to predict final results. With some preliminary data cleaning and feature enginering it create a csv file to be exported with the final results.

## Imported Libraries

In [118]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy as sp
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
import pickle

## Import the Test Data

In [119]:
df = pd.read_csv("kc_house_data_test_features.csv", index_col=0)
testing_df = df 
testing_df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


## Data Cleaning

In [120]:
testing_df.dtypes

id                 int64
date              object
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [121]:
#checking for null values
testing_df.isna().sum()

id               0
date             0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

In [122]:
#removing id column as its useless
#df.drop('id',axis=1,inplace=True)
testing_df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [123]:
#df['bedrooms'].value_counts()
#df['bathrooms'].value_counts()

In [124]:
#manipulating date column to add year, month, and day columns
testing_df['date'] = testing_df['date'].str.slice(0,8,1)
testing_df['year_sold'] = testing_df['date'].str.slice(0,4,1)
testing_df['month_sold'] = testing_df['date'].str.slice(4,6,1)
testing_df['day_sold'] = testing_df['date'].str.slice(6,8,1)

In [125]:
#changing the datatype of the date columns into integers
testing_df['year_sold'] = testing_df['year_sold'].astype('int64')
testing_df['month_sold'] = testing_df['month_sold'].astype('int64')
testing_df['day_sold'] = testing_df['day_sold'].astype('int64')

In [126]:
#removing date column
testing_df.drop('date',axis=1,inplace=True)

In [127]:
#testing_df.head()

In [128]:
zip_dummies = pd.get_dummies(testing_df['zipcode'],prefix='zip',drop_first=True)


In [129]:
#zip_dummies.head()

In [130]:
no_zip_df = testing_df.drop(['zipcode'], axis=1)

In [131]:
no_zip_df.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,year_sold,month_sold,day_sold
0,1974300020,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,47.7089,-122.241,2020,10918,2014,8,27
1,1974300020,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,47.7089,-122.241,2020,10918,2015,2,18
2,3630020380,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,47.5472,-121.998,1470,1576,2014,11,7
3,1771000290,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,47.7427,-122.071,1160,10565,2014,12,3
4,5126310470,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,47.4863,-122.140,2830,7916,2015,1,15


## Creating Dummy Variables

In [132]:
final_df = pd.concat([no_zip_df, zip_dummies], axis=1)

## Creating Polynomial interactions

In [133]:
poly_2 = PolynomialFeatures(degree=2, include_bias=False)
poly2_data = poly_2.fit_transform(final_df)
poly2_columns = poly_2.get_feature_names(final_df.columns)
df_poly2 = pd.DataFrame(poly2_data, columns=poly2_columns)
df_poly2.head()

id  bedrooms  bathrooms  sqft_living  sqft_lot  floors  \
0  1.974300e+09       4.0       2.50       2270.0   11500.0     1.0   
1  1.974300e+09       4.0       2.50       2270.0   11500.0     1.0   
2  3.630020e+09       3.0       2.50       1470.0    1779.0     2.0   
3  1.771000e+09       3.0       1.75       1280.0   16200.0     1.0   
4  5.126310e+09       4.0       2.75       2830.0    8126.0     2.0   

   waterfront  view  condition  grade  sqft_above  sqft_basement  yr_built  \
0         0.0   0.0        3.0    8.0      1540.0          730.0    1967.0   
1         0.0   0.0        3.0    8.0      1540.0          730.0    1967.0   
2         0.0   0.0        3.0    8.0      1160.0          310.0    2005.0   
3         0.0   0.0        3.0    8.0      1030.0          250.0    1976.0   
4         0.0   0.0        3.0    8.0      2830.0            0.0    2005.0   

   yr_renovated      lat     long  sqft_living15  sqft_lot15  year_sold  \
0           0.0  47.7089 -122.241         2020.0     10918.0     2014.0   
1           0.0  47.7089 -122.241         2020.0     10918.0     2015.0   
2           0.0  47.5472 -121.998         1470.0      1576.0     2014.0   
3           0.0  47.7427 -122.071         1160.0     10565.0     2014.0   
4           0.0  47.4863 -122.140         2830.0      7916.0     2015.0   

   month_sold  day_sold  zip_98002  zip_98003  zip_98004  zip_98005  \
0         8.0      27.0        0.0        0.0        0.0        0.0   
1         2.0      18.0        0.0        0.0        0.0        0.0   
2        11.0       7.0        0.0        0.0        0.0        0.0   
3        12.0       3.0        0.0        0.0        0.0        0.0   
4         1.0      15.0        0.0        0.0        0.0        0.0   

   zip_98006  zip_98007  zip_98008  zip_98010  zip_98011  zip_98014  \
0        0.0        0.0        0.0        0.0        0.0        0.0   
1        0.0        0.0        0.0        0.0        0.0        0.0   
2        0.0        0.0        0.0        0.0        0.0        0.0   
3        0.0        0.0        0.0        0.0        0.0        0.0   
4        0.0        0.0        0.0        0.0        0.0        0.0   

   zip_98019  zip_98022  zip_98023  zip_98024  zip_98027  zip_98028  \
0        0.0        0.0        0.0        0.0        0.0        0.0   
1        0.0        0.0        0.0        0.0        0.0        0.0   
2        0.0        0.0        0.0        0.0        0.0        0.0   
3        0.0        0.0        0.0        0.0        0.0        0.0   
4        0.0        0.0        0.0        0.0        0.0        0.0   

   zip_98029  zip_98030  zip_98031  zip_98032  zip_98033  zip_98034  \
0        0.0        0.0        0.0        0.0        0.0        1.0   
1        0.0        0.0        0.0        0.0        0.0        1.0   
2        1.0        0.0        0.0        0.0        0.0        0.0   
3        0.0        0.0        0.0        0.0        0.0        0.0   
4        0.0        0.0        0.0        0.0        0.0        0.0   

   zip_98038  zip_98039  zip_98040  zip_98042  zip_98045  zip_98052  \
0        0.0        0.0        0.0        0.0        0.0        0.0   
1        0.0        0.0        0.0        0.0        0.0        0.0   
2        0.0        0.0        0.0        0.0        0.0        0.0   
3        0.0        0.0        0.0        0.0        0.0        0.0   
4        0.0        0.0        0.0        0.0        0.0        0.0   

   zip_98053  zip_98055  zip_98056  zip_98058  zip_98059  zip_98065  \
0        0.0        0.0        0.0        0.0        0.0        0.0   
1        0.0        0.0        0.0        0.0        0.0        0.0   
2        0.0        0.0        0.0        0.0        0.0        0.0   
3        0.0        0.0        0.0        0.0        0.0        0.0   
4        0.0        0.0        0.0        0.0        1.0        0.0   

   zip_98070  zip_98072  zip_98074  zip_98075  zip_98077  zip_98092  \
0        0.0        0.0        0.

## Setting the holdout data to the relevant columns

In [134]:
holdout = df_poly2[['bedrooms', 'bathrooms', 'sqft_living', 'view', 'grade', 'sqft_above', 'sqft_basement', 'lat', 'sqft_living15', 'id sqft_living', 'id view', 'id sqft_above', 'bedrooms^2', 'bedrooms bathrooms', 'bedrooms sqft_living', 'bedrooms floors', 'bedrooms waterfront', 'bedrooms view', 'bedrooms grade', 'bedrooms sqft_above', 'bedrooms sqft_basement', 'bedrooms yr_built', 'bedrooms lat', 'bedrooms long', 'bedrooms sqft_living15', 'bedrooms year_sold', 'bathrooms^2', 'bathrooms sqft_living', 'bathrooms floors', 'bathrooms waterfront', 'bathrooms view', 'bathrooms condition', 'bathrooms grade', 'bathrooms sqft_above', 'bathrooms sqft_basement', 'bathrooms yr_built', 'bathrooms lat', 'bathrooms long', 'bathrooms sqft_living15', 'bathrooms year_sold', 'bathrooms month_sold', 'bathrooms zip_98004', 'sqft_living^2', 'sqft_living floors', 'sqft_living waterfront', 'sqft_living view', 'sqft_living condition', 'sqft_living grade', 'sqft_living sqft_above', 'sqft_living sqft_basement', 'sqft_living yr_built', 'sqft_living lat', 'sqft_living long', 'sqft_living sqft_living15', 'sqft_living year_sold', 'sqft_living month_sold', 'sqft_living day_sold', 'sqft_living zip_98004', 'floors view', 'floors grade', 'floors sqft_above', 'floors sqft_basement', 'floors sqft_living15', 'waterfront grade', 'waterfront sqft_above', 'waterfront sqft_basement', 'waterfront sqft_living15', 'view^2', 'view condition', 'view grade', 'view sqft_above', 'view sqft_basement', 'view yr_built', 'view lat', 'view long', 'view sqft_living15', 'view year_sold', 'view month_sold', 'view day_sold', 'condition grade', 'condition sqft_above', 'condition sqft_basement', 'condition sqft_living15', 'grade^2', 'grade sqft_above', 'grade sqft_basement', 'grade yr_built', 'grade lat', 'grade long', 'grade sqft_living15', 'grade year_sold', 'sqft_above^2', 'sqft_above sqft_basement', 'sqft_above yr_built', 'sqft_above lat', 'sqft_above long', 'sqft_above sqft_living15', 'sqft_above year_sold', 'sqft_above month_sold', 'sqft_above day_sold', 'sqft_above zip_98004', 'sqft_basement^2', 'sqft_basement yr_built', 'sqft_basement lat', 'sqft_basement long', 'sqft_basement sqft_living15', 'sqft_basement year_sold', 'yr_built sqft_living15', 'lat^2', 'lat sqft_living15', 'long sqft_living15', 'sqft_living15^2', 'sqft_living15 year_sold', 'sqft_living15 month_sold', 'sqft_living15 zip_98004']]

## Importing my model and scaler

In [135]:
scaler = pickle.load(open('scaler2.pickle', 'rb'))
final_model = pickle.load(open('finalized_model.sav', 'rb'))

## Scalling and fitting the holdout Data

In [136]:
scaler2.fit(holdout)
holdout = pd.DataFrame(data=scaler2.transform(holdout), columns=holdout.columns)

In [137]:
final_answer = final_model.predict(holdout)

## Getting Predictions

In [138]:
final_answer = pd.DataFrame(final_answer, columns=['predictions'])

In [139]:
final_answer.head()

,predictions
0,575797.730130
1,606113.350179
2,418205.707609
3,442375.772983
4,561862.791472


## Exporting the CSV

In [140]:
final_answer.to_csv('housing_predictions_Kelvin_Arellano.csv')